In [ ]:
import sys, os

if 'COLAB_GPU' in os.environ:
    print("Running on Google Colab, setting up environment")
    colab = True
    !pip install scikit-video

    !mkdir -p ./src/data  
    !mkdir -p ./src/modules  
    !mkdir -p ./data
    !mkdir -p ./models

    !wget -P ./src/data https://raw.githubusercontent.com/quietscientist/Infant_movement_assessment/master/src/data/build_features.py
    !wget -P ./src/data https://raw.githubusercontent.com/quietscientist/Infant_movement_assessment/master/src/data/compute_surprise.py
    !wget -P ./src/data https://raw.githubusercontent.com/quietscientist/Infant_movement_assessment/master/src/data/load_pose_data.py
    !wget -P ./src/data https://raw.githubusercontent.com/quietscientist/Infant_movement_assessment/master/src/data/merge_data_sets.py
    !wget -P ./src/data https://raw.githubusercontent.com/quietscientist/Infant_movement_assessment/master/src/data/preprocess_pose_data.py


    !wget -P  ./src/modules https://raw.githubusercontent.com/quietscientist/Infant_movement_assessment/master/src/modules/circstats.py
    !wget -P  ./src/modules https://raw.githubusercontent.com/quietscientist/Infant_movement_assessment/master/src/modules/skeleton.py
    !wget -P  ./src/modules https://raw.githubusercontent.com/quietscientist/Infant_movement_assessment/master/src/modules/util_data.py
    !wget -P  ./src/modules https://raw.githubusercontent.com/quietscientist/Infant_movement_assessment/master/src/modules/util_vis.py

    sys.path.insert(0,'./src/data')
    sys.path.insert(0,'./src/modules')

    data_path = './data'
    model_path = './models'
    home_path = './'

else: 
    colab = False
    sys.path.insert(0,'../src/data')
    sys.path.insert(0,'../src/modules')

    data_path = '../data'
    model_path = '../models'
    home_path = '../'

In [ ]:
import pandas as pd
from scipy.stats import norm
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec


import load_pose_data
import skvideo.io
import itertools
from sklearn.metrics import roc_auc_score, roc_curve
from scipy.linalg import svd
import preprocess_pose_data, build_features, merge_data_sets, compute_surprise
import  util_vis
from util_vis import plot_skel, gen_one_frame, get_one_frame, normm

from skeleton import Skeleton

In [ ]:
def setup_env():
    if not os.path.exists(data_path):
        os.makedirs(data_path)

    if not os.path.exists(model_path):
        os.makedirs(model_path)

    #if data directory is empty, rerun setup 
    if not os.path.exists(data_path) or len(os.listdir(data_path)) == 0:

        if colab:
            !wget https://figshare.com/ndownloader/files/15593963?private_link=10034c230ad9b2b2a6a4 -O ./pose_data.zip
            !unzip -o ./pose_data.zip -d ./


            !mv --force ./infant_movement_assessment_repo_files/data/* ./data
            !mv --force ./infant_movement_assessment_repo_files/models/* ./models

            !rm ./pose_data.zip
            !rm -r ./infant_movement_assessment_repo_files

        else:
            !wget https://figshare.com/ndownloader/files/15593963?private_link=10034c230ad9b2b2a6a4 -O ../pose_data.zip
            !unzip -o ../pose_data.zip -d ../


            !mv --force ../infant_movement_assessment_repo_files/data/* ../data
            !mv --force ../infant_movement_assessment_repo_files/models/* ../models

            !rm ../pose_data.zip
            !rm -r ../infant_movement_assessment_repo_files
    else: 
        print('setup for this project has already been done. Skipping download and extraction of data.')

def compute_kinematics(datasets, data_path):
    for data_set in data_sets:
        print(f'Processing {data_set} data set')

        raw_pose_estimates_video_path = f'{data_path}/pose_estimates/{data_set}/py/pose_estimates.pkl'
        assert os.path.exists(raw_pose_estimates_video_path), 'Folder does not exist. \n Edit path to pose estimate data (raw_pose_estimates_video_path) in notebooks/infant_move/master.ipynb, Cell 2.'

        #load_pose_data.main(data_set, raw_pose_estimates_video_path)
        preprocess_pose_data.main(data_set, data_path)
        build_features.main(data_set, data_path)


In [ ]:
def convert_coco_to_openpose(coco_keypoints):
    """
    Convert COCO keypoints to OpenPose keypoints with the neck keypoint as the midpoint between the two shoulders.
    COCO keypoints format (17 keypoints): [nose, left_eye, right_eye, left_ear, right_ear,
                                           left_shoulder, right_shoulder, left_elbow, right_elbow,
                                           left_wrist, right_wrist, left_hip, right_hip,
                                           left_knee, right_knee, left_ankle, right_ankle]
    OpenPose keypoints format (18 keypoints): COCO keypoints + [neck]
    The neck is not a part of COCO keypoints and is computed as the midpoint between the left and right shoulders.
    """

    # Assuming coco_keypoints is a list of (x, y) tuples
    nose, left_eye, right_eye, left_ear, right_ear, \
    left_shoulder, right_shoulder, left_elbow, right_elbow, \
    left_wrist, right_wrist, left_hip, right_hip, \
    left_knee, right_knee, left_ankle, right_ankle = coco_keypoints

    # Calculate the neck as the midpoint between left_shoulder and right_shoulder
    neck_x = (left_shoulder[0] + right_shoulder[0]) / 2
    neck_y = (left_shoulder[1] + right_shoulder[1]) / 2
    neck = (neck_x, neck_y)

    # Construct the OpenPose keypoints including the neck
    openpose_keypoints = [
        nose, left_eye, right_eye, left_ear, right_ear,
        left_shoulder, right_shoulder, left_elbow, right_elbow,
        left_wrist, right_wrist, left_hip, right_hip,
        left_knee, right_knee, left_ankle, right_ankle,
        neck  # Adding the neck as the last keypoint
    ]

    return openpose_keypoints

def reorder_keypoints(keypoints):
    """
    Reorder the keypoints to the OpenPose format.
    The OpenPose format is as follows:
    0-17: [nose, neck, right_shoulder, right_elbow, right_wrist, left_shoulder, left_elbow, left_wrist,
           right_hip, right_knee, right_ankle, left_hip, left_knee, left_ankle, right_eye, left_eye, right_ear, left_ear]
    The input 'keypoints' is a list of (x, y, c) tuples, where c is the confidence score.
    """

    # Reorder the keypoints to the OpenPose format
    keypoints = [keypoints[i] for i in [0, 17, 6, 8, 10, 5, 7, 9, 12, 14, 16, 11, 13, 15, 2, 1, 4, 3]]

    return keypoints

def rescale_keypoints(keypoints, scale):
    """
    Rescale the keypoints by the given scale.
    The input 'keypoints' is a list of (x, y) tuples
    """

    # Rescale the keypoints
    keypoints = [(x * scale, y * scale) for (x, y) in keypoints]

    return keypoints